# Plotting routine for the Sep22 Event

In [1]:
test = 'test'

In [2]:
%load_ext autoreload

%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
import pickle as p

import datetime as datetime
from datetime import timedelta
import seaborn as sns
import pandas as pd
import os
#import copy as cp

import multiprocessing
import py3dcore_h4c as py3dcore_h4c

import py3dcore_h4c.fluxplot as fp

plt.switch_backend('agg')


import warnings
warnings.filterwarnings('ignore')

import logging

logging.basicConfig(level=logging.INFO)
logging.getLogger("heliosat.spice").setLevel("WARNING")
logging.getLogger("heliosat.spacecraft").setLevel("WARNING")
#logging.getLogger("sunpy").setLevel("WARNING")
#logging.getLogger("py3dcore_h4c.fluxplot").setLevel("WARNING")

#print(sns.__version__)

#### setting the times for py3DCORE_h4c

In [3]:
t_launch = datetime.datetime(2022, 9, 5, 16, tzinfo=datetime.timezone.utc) # 

t_s = datetime.datetime(2022, 9, 5, 18, tzinfo=datetime.timezone.utc) 
t_e = datetime.datetime(2022, 9, 6, 1, tzinfo=datetime.timezone.utc)

t_fit = [
    datetime.datetime(2022, 9, 5, 18, 30, tzinfo=datetime.timezone.utc),
    datetime.datetime(2022, 9, 5, 20, tzinfo=datetime.timezone.utc),
    datetime.datetime(2022, 9, 5, 22, tzinfo=datetime.timezone.utc),
    datetime.datetime(2022, 9, 6, 0, tzinfo=datetime.timezone.utc)
]

#t_fit = fp.equal_t_creator(start = start, n = 4, delta = 6)

#####2022 Sep 5: 3 solar radii at 17 UT (COR2 & LASCO FoV appearance)
# 2022 Sep 5: 15 solar radii at 18:45 UT (PSP)

#### loading the pickle files

In [4]:
t_fit

[datetime.datetime(2022, 9, 5, 18, 30, tzinfo=datetime.timezone.utc),
 datetime.datetime(2022, 9, 5, 20, 0, tzinfo=datetime.timezone.utc),
 datetime.datetime(2022, 9, 5, 22, 0, tzinfo=datetime.timezone.utc),
 datetime.datetime(2022, 9, 6, 0, 0, tzinfo=datetime.timezone.utc)]

In [5]:
path = 'output/psp06092022_heeq_512_1/'

filepath = fp.loadpickle(path, number = -1)

In [6]:
filepath[:-7]

'output/psp06092022_heeq_512_1/13'

#### extracting parameters

In [48]:
res, allres, ind, meanparams = fp.get_params(filepath, give_mineps=True)

INFO:py3dcore_h4c.fluxplot:Retrieved the following parameters for the run with minimum epsilon:
INFO:py3dcore_h4c.fluxplot: --lon:  76.04
INFO:py3dcore_h4c.fluxplot: --lat:  -19.30
INFO:py3dcore_h4c.fluxplot: --inc:  6.73
INFO:py3dcore_h4c.fluxplot: --diameter 1 AU:  0.29
INFO:py3dcore_h4c.fluxplot: --aspect ratio:  1.40
INFO:py3dcore_h4c.fluxplot: --launch radius:  13.03
INFO:py3dcore_h4c.fluxplot: --launch speed:  1976.00
INFO:py3dcore_h4c.fluxplot: --t factor:  192.63
INFO:py3dcore_h4c.fluxplot: --expansion rate:  1.14
INFO:py3dcore_h4c.fluxplot: --magnetic field decay rate:  1.64
INFO:py3dcore_h4c.fluxplot: --magnetic field 1 AU:  8.51
INFO:py3dcore_h4c.fluxplot: --drag coefficient:  1.87
INFO:py3dcore_h4c.fluxplot: --sw background speed:  100.39


In [49]:
out = fp.get_ensemble_stats(filepath)

In [50]:
out.describe()

,lon,lat,inc,D1AU,delta,launch radius,launch speed,t factor,B1AU,gamma,vsw
count,512.000000,512.000000,512.000000,512.000000,512.000000,512.000000,512.000000,512.000000,512.000000,512.000000,512.000000
mean,73.547287,-16.260410,3.790438,0.298286,1.322295,13.331536,1790.229858,195.726044,7.481790,1.637771,152.038818
std,1.770907,2.121543,1.575535,0.024677,0.216009,0.484476,148.199402,27.670771,1.052507,0.266650,47.887653
min,68.451485,-19.956299,0.000186,0.227883,1.002483,11.716013,1272.493896,119.413979,5.019146,0.698808,100.158112
25%,72.284582,-17.884840,2.664175,0.281472,1.151131,13.060473,1706.376404,177.309181,6.692626,1.452294,116.262392
50%,73.726570,-16.430988,3.828349,0.299065,1.288844,13.441062,1818.738586,198.924965,7.465033,1.682794,136.485886
75%,74.865583,-14.883627,4.892369,0.316381,1.456471,13.720273,1907.054382,216.613392,8.162389,1.856927,176.718437
max,77.764130,-9.209253,8.172051,0.349891,2.192111,13.999943,1999.890747,249.016830,11.047436,1.999986,380.915619


#### loading and plotting the fitting results

In [8]:
fp.fullinsitu(observer='PSP', t_fit=t_fit, start=t_s, end=t_e, filepath=filepath, 
              custom_data=False, best=True, ensemble=True, mean=False, save_fig=False, legend=False, fixed=None)

INFO:py3dcore_h4c.fluxplot:Using HelioSat to retrieve observer data
INFO:py3dcore_h4c.fluxplot:No iparams_arr given, using parameters for run with minimum eps.
INFO:py3dcore_h4c.fitter.base:Using HelioSat to retrieve observer data


In [11]:
fp.scatterparams(filepath)

In [12]:
print(out['lon'].mean())
print(out['lon'].std())
print(out['launch radius'].std())
print(out['launch speed'].std())

172.78905
4.882709
0.53428584
266.25516


#### Plotting the fluxrope and spacecraft positions

In [9]:
#measurement times 
#tm0 =  t_launch + datetime.timedelta(days=1.5)
tm0 =  datetime.datetime(2022, 9, 7, 1)
tm1 =  t_launch + datetime.timedelta(days=1.7)
tm2 =  t_launch + datetime.timedelta(days=3.5)

fig = fp.full3d(spacecraftlist=['solo', 'psp'], planetlist =['Earth'],t = tm1, filepath = filepath)

INFO:sunpy:Obtained JPL HORIZONS location for Solar Orbiter (spacecraft) (-144


INFO: Obtained JPL HORIZONS location for Solar Orbiter (spacecraft) (-144 [sunpy.coordinates.ephemeris]


INFO:py3dcore_h4c.fluxplot:Indices of date: 59
INFO:py3dcore_h4c.fluxplot:Solar Orbiter - r: 0.684483, lon: 150.072220, lat: -3.730396, 
INFO:sunpy:Obtained JPL HORIZONS location for Solar Orbiter (spacecraft) (-144
INFO:py3dcore_h4c.fluxplot:Indices of date: 35
INFO:py3dcore_h4c.fluxplot:Solar Orbiter - r: 0.684483, lon: 150.072220, lat: -3.730396, 
INFO:sunpy:Obtained JPL HORIZONS location for Parker Solar Probe (spacecraft)


INFO: Obtained JPL HORIZONS location for Solar Orbiter (spacecraft) (-144 [sunpy.coordinates.ephemeris]
INFO: Obtained JPL HORIZONS location for Parker Solar Probe (spacecraft) [sunpy.coordinates.ephemeris]


INFO:py3dcore_h4c.fluxplot:Indices of date: 59
INFO:py3dcore_h4c.fluxplot:Parker Solar Probe - r: 0.095459, lon: -122.155481, lat: -2.099772, 
INFO:sunpy:Obtained JPL HORIZONS location for Parker Solar Probe (spacecraft)
INFO:py3dcore_h4c.fluxplot:Indices of date: 35
INFO:py3dcore_h4c.fluxplot:Parker Solar Probe - r: 0.095459, lon: -122.155481, lat: -2.099772, 


INFO: Obtained JPL HORIZONS location for Parker Solar Probe (spacecraft) [sunpy.coordinates.ephemeris]


### make fluxrope 3d movie

In [14]:
def make_frame(k):
    tm0 = datetime.datetime(2022, 9, 5, 19)
    t = tm0 + k*datetime.timedelta(hours=1)
    
    fig = fp.full3d(spacecraftlist=['solo', 'psp'], planetlist =['Earth'],
              t = t, filepath = filepath, save_fig = False)
    frmstr = '%05i' % (k) 
    plt.savefig(filepath[:-7] + 'movie_3d/'+frmstr+'.jpg',dpi=200)  
    return fig
    

In [15]:
inn=[i for i in range(100)]


mpool = multiprocessing.Pool(processes=5)
mpool.map(make_frame, inn[0:20])
mpool.close()
mpool.join()

print('done 1/4')

mpool = multiprocessing.Pool(processes=5)
mpool.map(make_frame, inn[20:40])
mpool.close()
mpool.join()

print('done 2/4')

mpool = multiprocessing.Pool(processes=5)
mpool.map(make_frame, inn[40:60])
mpool.close()
mpool.join()

print('done 3/4')

mpool = multiprocessing.Pool(processes=5)
mpool.map(make_frame, inn[40:80])
mpool.close()
mpool.join()

print('done 4/4')

done 1/4
done 2/4
done 3/4
done 4/4


In [16]:
os.system('ffmpeg -r 25 -i '+filepath[:-7]+'movie_3d/%05d.jpg -b 5000k -r 25 '+filepath[:-7]+'movie_3d/full_3d_movie.mp4 -y -loglevel quiet')

0

### plot fluxrope 3d multiview

In [17]:
#measurement times 
#tm0 =  t_launch + datetime.timedelta(days=1.5)
tm0 =  datetime.datetime(2022, 9, 7, 1)
tm1 =  t_launch + datetime.timedelta(days=1.7)
tm2 =  t_launch + datetime.timedelta(days=3.5)

fig = fp.full3d_multiview(t_launch = t_launch, filepath=filepath)

### make fluxrope 3d multiview movie

In [18]:
def make_frame2(k):
    tm0 = datetime.datetime(2022, 9, 5, 19)
    t = tm0 + k*datetime.timedelta(hours=1)
    frametime = k
    
    fig = fp.full3d_multiview_movie(t_launch = tm0, t = t, filepath=filepath,
                                    frametime=k)
    frmstr = '%05i' % (k) 
    plt.savefig(filepath[:-7] + 'movie_3d_multiview/'+frmstr+'.jpg',dpi=200)  
    return fig

In [19]:
inn=[i for i in range(100)]


mpool = multiprocessing.Pool(processes=5)
mpool.map(make_frame2, inn[0:20])
mpool.close()
mpool.join()

print('done 1/4')

mpool = multiprocessing.Pool(processes=5)
mpool.map(make_frame2, inn[20:40])
mpool.close()
mpool.join()

print('done 2/4')

mpool = multiprocessing.Pool(processes=5)
mpool.map(make_frame2, inn[40:60])
mpool.close()
mpool.join()

print('done 3/4')

mpool = multiprocessing.Pool(processes=5)
mpool.map(make_frame2, inn[40:80])
mpool.close()
mpool.join()

print('done 4/4')

The history saving thread hit an unexpected error (OperationalError('disk I/O error')).History will not be written to the database.
done 1/4
done 2/4
done 3/4
done 4/4


In [20]:
os.system('ffmpeg -r 25 -i '+filepath[:-7]+'movie_3d_multiview/%05d.jpg -b 5000k -r 25 '+filepath[:-7]+'movie_3d/full_3d_multiview_movie.mp4 -y -loglevel quiet')

0

# Used for Debug

In [21]:
TP_A =  t_launch + datetime.timedelta(hours=2)
TP_B =  t_launch + datetime.timedelta(hours=40)


C_A = "xkcd:red"
C_B = "xkcd:blue"
    
sns.set_style('whitegrid')

fig = plt.figure(figsize=(15, 11),dpi=100)

ax1 = plt.subplot2grid((2, 3), (0, 0),rowspan=2,colspan=2,projection='3d')  
ax2 = plt.subplot2grid((2, 3), (0, 2),projection='3d')  
ax3 = plt.subplot2grid((2, 3), (1, 2),projection='3d')  

model_obj = fp.returnmodel(filepath)


fp.plot_configure(ax1, view_azim=150, view_elev=25, view_radius=.2,light_source=True) #view_radius=.08

fp.plot_3dcore(ax1, model_obj, TP_A, color=C_A,light_source = True)
fp.plot_3dcore(ax1, model_obj, TP_B, color=C_B,light_source = True)

fp.plot_shift(ax1,0.31,-0.25,0.0,-0.2)


fp.plot_configure(ax2, view_azim=165-90, view_elev=90, view_radius=.08,light_source=True)
    
fp.plot_3dcore(ax2, model_obj, TP_A, color=C_A,light_source = True)
fp.plot_3dcore(ax2, model_obj, TP_B, color=C_B,light_source = True)

fp.plot_shift(ax2,0.26,-0.41,0.08,0.0)



fp.plot_configure(ax3, view_azim=65, view_elev=-5, view_radius=.01,light_source=True)
    
fp.plot_3dcore(ax3, model_obj, TP_A, color=C_A,light_source = True)
fp.plot_3dcore(ax3, model_obj, TP_B, color=C_B,light_source = True)

fp.plot_shift(ax3,0.26,-0.41,0.08,0.0)

